In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',None)
train_identity = pd.read_csv('Data/train_identity.csv')
train_transaction = pd.read_csv('Data/train_transaction.csv')

In this notebook, we'll look at the predictive power of different variables from the identity and transaction data. To do that, we'll first need to convert the categorical variables into dummies and remove NaNs.

In [2]:
#Separating identity data into strings and numerics
identity_categorical = train_identity.select_dtypes(include='object')
identity_numerics = train_identity.select_dtypes(exclude='object')

#Getting dummies and joining back toegether with numerics
identity_dummies = pd.get_dummies(identity_categorical, dummy_na = True)
train_identity = pd.concat([identity_numerics, identity_dummies], axis=1)

#Filling NaNs with mean
train_identity = train_identity.fillna(train_identity.mean())

#Separating transaction data into strings and numerics
transaction_categorical = train_transaction.select_dtypes(include='object')
transaction_numerics = train_transaction.select_dtypes(exclude='object')

#Getting dummies and joining back toegether with numerics
transaction_dummies = pd.get_dummies(transaction_categorical, dummy_na = True)
train_transaction = pd.concat([transaction_numerics, transaction_dummies], axis=1)

#Filling NaNs with mean
train_transaction = train_transaction.fillna(train_transaction.mean())

In [3]:
#Joining fraud data with identity data

train_identity = train_identity.merge(train_transaction[['TransactionID','isFraud']], on='TransactionID')

In [4]:
train_identity.head()

TransactionID  id_01     id_02     id_03     id_04     id_05    id_06  \
0        2987004    0.0   70787.0  0.060189 -0.058938  1.615585 -6.69871   
1        2987008   -5.0   98945.0  0.060189 -0.058938  0.000000 -5.00000   
2        2987010   -5.0  191631.0  0.000000  0.000000  0.000000  0.00000   
3        2987011   -5.0  221832.0  0.060189 -0.058938  0.000000 -6.00000   
4        2987016    0.0    7460.0  0.000000  0.000000  1.000000  0.00000   

       id_07      id_08     id_09     id_10  id_11      id_13       id_14  \
0  13.285354 -38.600388  0.091023 -0.301124  100.0  48.053071 -480.000000   
1  13.285354 -38.600388  0.091023 -0.301124  100.0  49.000000 -300.000000   
2  13.285354 -38.600388  0.000000  0.000000  100.0  52.000000 -344.507146   
3  13.285354 -38.600388  0.091023 -0.301124  100.0  52.000000 -344.507146   
4  13.285354 -38.600388  0.000000  0.000000  100.0  48.053071 -300.000000   

   id_17      id_18  id_19  id_20      id_21      id_22      id_24  \
0  166.0  14.237337  542.0  144.0  368.26982  16.002708  12.800927   
1  166.0  14.237337  621.0  500.0  368.26982  16.002708  12.800927   
2  121.0  14.237337  410.0  142.0  368.26982  16.002708  12.800927   
3  225.0  14.237337  176.0  507.0  368.26982  16.002708  12.800927   
4  166.0  15.000000  529.0  575.0  368.26982  16.002708  12.800927   

        id_25       id_26      id_32  id_12_Found  id_12_NotFound  id_12_nan  \
0  329.608924  149.070308  32.000000            0               1          0   
1  329.608924  149.070308  32.000000            0               1          0   
2  329.608924  149.070308  26.508597            0               1          0   
3  329.608924  149.070308  26.508597            0               1          0   
4  329.608924  149.070308  24.000000            0               1          0   

   id_15_Found  id_15_New  id_15_Unknown  id_15_nan  id_16_Found  \
0            0          1              0          0            0   
1            0          1              0          0            0   
2            1          0              0          0            1   
3            0          1              0          0            0   
4            1          0              0          0            1   

   id_16_NotFound  id_16_nan  id_23_IP_PROXY:ANONYMOUS  id_23_IP_PROXY:HIDDEN  \
0               1          0                         0                      0   
1               1          0                         0                      0   
2               0          0                         0                      0   
3               1          0                         0                      0   
4               0          0                         0                      0   

   id_23_IP_PROXY:TRANSPARENT  id_23_nan  id_27_Found  id_27_NotFound  \
0                           0          1            0               0   
1                           0          1            0               0   
2                           0          1            0               0   
3                           0          1            0               0   
4                           0          1            0               0   

   id_27_nan  id_28_Found  id_28_New  id_28_nan  id_29_Found  id_29_NotFound  \
0          1            0          1          0            0               1   
1          1            0          1          0            0               1   
2          1            1          0          0            1               0   
3          1            0          1          0            0               1   
4          1            1          0          0            1               0   

   id_29_nan  id_30_Android  id_30_Android 4.4.2  id_30_Android 5.0  \
0          0              0                    0                  0   
1          0              0                    0                  0   
2          0              0                    0                  0   
3          0              0                    0                  0   
4         

Looking at correlations of identity variables with fraud variable

In [5]:
identity_corrs = train_identity.loc[:,train_identity.columns != 'isFraud'].corrwith(train_identity['isFraud'])
print(abs(identity_corrs).sort_values(ascending=False))

id_17                                       0.148223
id_35_F                                     0.144881
id_30_nan                                   0.136758
id_35_T                                     0.135995
id_34_nan                                   0.135543
id_33_nan                                   0.123155
id_34_match_status:2                        0.122979
id_01                                       0.120099
DeviceInfo_hi6210sft Build/MRA58K           0.117353
DeviceInfo_SM-A300H Build/LRX22G            0.105271
id_29_Found                                 0.100502
id_16_NotFound                              0.099017
id_16_Found                                 0.098682
id_28_Found                                 0.095606
id_15_New                                   0.093855
id_15_Found                                 0.093164
TransactionID                               0.092935
id_29_NotFound                              0.092740
id_28_New                                   0.

Now moving onto correlations with transaction variables

In [6]:
transaction_corrs = train_transaction.iloc[:,2:].corrwith(train_transaction['isFraud'])
print(abs(transaction_corrs).sort_values(ascending=False))

V257                            0.262946
V246                            0.251838
V244                            0.249951
V242                            0.247522
V45                             0.236688
V201                            0.234520
V200                            0.227926
V86                             0.224530
V87                             0.224450
V189                            0.220374
V44                             0.218669
V188                            0.217058
V258                            0.203975
V52                             0.201111
V51                             0.187440
V228                            0.184556
V170                            0.178601
V40                             0.178413
V79                             0.173097
V39                             0.170565
V94                             0.167984
V38                             0.167128
V43                             0.166514
V33                             0.165534
V199            

Let's try using a different metric. Here I'm going to run a logistic regression of each variable on the fraud dependent variable and evaluate its predictions using the roc_auc_score

In [7]:
from sklearn.metrics import roc_auc_score
from sklearn import model_selection as ms
from sklearn.metrics import make_scorer
from sklearn.linear_model import LogisticRegression

In [11]:
cols = abs(transaction_corrs).sort_values(ascending=False).index
fraud_log = LogisticRegression(solver='sag')

scores = []
for col in cols:
    fraud_log_fit = fraud_log.fit(X=train_transaction[col].values.reshape(-1, 1),y=train_transaction['isFraud'])
    preds = fraud_log_fit.predict_proba(X=train_transaction[col].values.reshape(-1, 1))
    score = roc_auc_score(train_transaction['isFraud'], preds[:,1])
    scores.append(score)

scores = pd.DataFrame(data=scores, index=cols)

print(scores)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_

                                     0
V257                          0.574209
V246                          0.552368
V244                          0.524271
V242                          0.520409
V45                           0.627514
V201                          0.550267
V200                          0.537056
V86                           0.621585
V87                           0.631148
V189                          0.499443
V44                           0.614191
V188                          0.485931
V258                          0.591857
V52                           0.683776
V51                           0.680958
V228                          0.546532
V170                          0.557720
V40                           0.670627
V79                           0.681203
V39                           0.669084
V94                           0.687749
V38                           0.617593
V43                           0.666468
V33                           0.681449
V199                     

In [14]:
scores

,0
V257,0.574209
V246,0.552368
V244,0.524271
V242,0.520409
V45,0.627514
V201,0.550267
V200,0.537056
V86,0.621585
V87,0.631148
V189,0.499443
